In [ ]:
#### Start in KI_Compartment root folder
# You must have SymITS2 and all its dependencies installed and working from your home folder (i.e. at ~/SymITS2)
# You must have a working version of QIIME, with UCLUST and USEARCH61 installed.
# You must have the nucleotide NCBI blast database locally on your machine, and a working version of blast
    # You might want to put this on a hard drive, as it is currently ~94GB of data (as of August 1, 2017)
    # Download Blastdb from ftp://ftp.ncbi.nlm.nih.gov/blast/documents/blastdb.html, dataset: nt.*tar.gz.
    # Then make sure to unzip all folders and place in one folder called "blastdb"
    # Mine is at "/media/danielle/CoralDrive/blastdb"
    # For getting blast installed on Linux Ubuntu, this website was helpful: http://www.blopig.com/blog/2014/04/quick-standalone-blast-setup-for-ubuntu-linux/
# You must have an updated version of R, I used R 3.4.1 (2017-06-30, Single Candle)
    # Required R packages are specified in SymITS2

# Prep raw data
#chmod u+x data/Bioinf/extract_unzip_prep_rawdata.sh # Make sure script can run
#data/Bioinf/extract_unzip_prep_rawdata.sh # This does the following:
    # Unzips all original files
    # Makes configs for Bokulich qc
    # Runs Bokulich qc
    # Makes configs for Illumina paired end merging
    # Merges Illumina pairs, includes Q30 check
    # Filters merged reads
    
# Or run each command from extract_unzip_prep_rawdata individually...
find . -type f -exec gunzip {} + # Unzip these files
# They are going to get really big! And this may take quite some time!

python data/Bioinf/make_configs.py # Make the Illumina-utils config files for the Bokulich QC method

python data/Bioinf/boku_qc.py # Run Bokulich QC method on all applicable files
chmod u+x data/Bioinf/boku_qc_KI_Compartment.sh # Give permissions so .sh file will run

source ~/virtual-envs/illumina-utils-v2.0.0/bin/activate # Activate Illumina-utils
data/Bioinf/boku_qc_KI_Compartment.sh # Run .sh file that was just created 

python data/Bioinf/make_merge_configs.py # Make the Illumina-utils config files for merging pairs

python data/Bioinf/iu_merge_pairs.py # Merge pairs using Illumina-utils
chmod u+x data/Bioinf/iu-merge-pairs_KI_Compartment.sh # Give permissions so .sh file will run
data/Bioinf/iu-merge-pairs_KI_Compartment.sh # Run .sh file that was just created 
# find . -maxdepth 1 -name # To check how far along the merging is (because it takes a long time to do hundreds of samples)

python data/Bioinf/iu_filter_merged_reads.py # Filter merged reads (MAX-MISMATCH=3) using Illumina-utils
chmod u+x data/Bioinf/iu-filter-merged-reads_KI_Compartment.sh # Give permissions so .sh file will run
data/Bioinf/iu-filter-merged-reads_KI_Compartment.sh # Run .sh file that was just created 

python data/Bioinf/rename.py # Rename merged files for downstream processing

In [ ]:
# Now, because I named these samples unneccesarily differently when I sent them in for sequencing, let's fix that before moving downstream...

rename 's/DClaar_2014_S_\.*/KI14SSYM0/' *.fasta
rename 's/DClaar_2014_W_\.*/KI14WSYM0/' *.fasta
rename 's/DClaar_2015a_S_\.*/KI15aSSYM0/' *.fasta
rename 's/DClaar_2015a_W_\.*/KI15aWSYM0/' *.fasta
rename 's/DClaar_2015b_W_\.*/KI15bWSYM0/' *.fasta
rename 's/DClaar_2015b_S_\.*/KI15bSSYM0/' *.fasta


In [1]:
# Next, start using SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
~/SymITS2/qc_trim_reads.sh # This does the following:
    # Adds QIIME labels
    # Identifies chimeric sequences
    # Filters out chimeric sequences
    # Runs cutadapt 4 times
    # Removes intermediate files from cutadapt process

Error in add_qiime_labels.py: option -i: directory does not exist: 'data/merge/'

If you need help with QIIME, see:
http://help.qiime.org
Error in identify_chimeric_seqs.py: option -i: file does not exist: 'data/fasta/combined_seqs.fna'

If you need help with QIIME, see:
http://help.qiime.org
Error in filter_fasta.py: option -f: file does not exist: 'data/fasta/combined_seqs.fna'

If you need help with QIIME, see:
http://help.qiime.org
cutadapt version 1.14
Copyright (C) 2010-2017 Marcel Martin <marcel.martin@scilifelab.se>

cutadapt removes adapter sequences from high-throughput sequencing reads.

Usage:
    cutadapt -a ADAPTER [options] [-o output.fastq] input.fastq

For paired-end reads:
    cutadapt -a ADAPT1 -A ADAPT2 [options] -o out1.fastq -p out2.fastq in1.fastq in2.fastq

Replace "ADAPTER" with the actual sequence of your 3' adapter. IUPAC wildcard
characters are supported. The reverse complement is *not* automatically
searched. All reads from input.fastq will be written to ou

: 1

In [2]:
count_seqs.py -i data/fasta/combined_seqs_trimmed.fasta # Count how many seqs there are after quality filtering


7332473  : data/fasta/combined_seqs_trimmed.fasta (Sequence lengths (mean +/- std): 306.5629 +/- 24.1155)
7332473  : Total


In [ ]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Cluster at 97% within samples (each sample is clustered independently)
~/SymITS2/otus_97_bysample.sh data/fasta/combined_seqs_trimmed.fasta data/Bioinf/clust
# Arguments are 1) Combined, trimmed sequences (fasta file) and 2)
Output directory
# Creates data/Bioinf/clust/all_rep_set_rep_set.fasta for downstream use

When using programs that use GNU Parallel to process data for publication please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; and it won't cost you a cent.
Or you can get GNU Parallel without this requirement by paying 10000 EUR.

To silence this citation notice run 'parallel --bibtex' once or use '--no-notice'.

When using programs that use GNU Parallel to process data for publication please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; and it won't cost you a cent.
Or you can get GNU Parallel without this requirement by paying 10000 EUR.

To silence this citation notice run 'parallel --bibtex' once or use '--no-notice'.



In [4]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Use global alignment to identify sequences
R --vanilla < ~/SymITS2/run_nw.R --args data/Bioinf/clust/all_rep_set_rep_set.fasta data/ITS2db_trimmed_derep.fasta
# Arguments are 1) query sequences - rep set fasta file, and 2) reference database sequences in .fasta format
# Creates data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv for downstream use


R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # Get command line arguments
> args = commandArgs(trailingOnly=TRUE)
> # If two arguments not provided, return an error
> if (length(args) < 2) {
+   stop("must specify query sequences (.fasta) and reference sequences (.fasta)", call.=FALSE)
+ }
> 
> # Import query sequences
> library(Biostrings)
Loading req

: 1

In [ ]:
# Now extract metadata from full metadata spreadsheet
# Using mapping_file.txt
R --vanilla < data/Coralphoto__Metadata/process_coral_metadata.R 


In [ ]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Build a phyloseq object
R --vanilla < ~/SymITS2/build_phyloseq.R --args data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv data/Coralphoto__Metadata/KI_Compartment_metadata.tsv data/Bioinf/clust/97_otus_bysample.tsv data/ITS2db_trimmed_notuniques_otus.txt analyses/KI_Compartment.RData
# Arguments are 1) nw_tophits - taxonomic assignment output from run_nw.R, 2) sample metadata in tsv format, 3) OTU table, 4) duplicate reference taxa names, 5) output filename
# Creates analyses/KI_Compartment.RData

In [ ]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Filter out sequences which do not blast to Symbiodnium
R --vanilla < ~/SymITS2/filter_notsym.R --args analyses/KI_Compartment.RData data/Bioinf/clust/all_rep_set_rep_set.fasta analyses/KI_Compartment.RData /media/danielle/CoralDrive/blastdb
# Output file is KI_Compartment.RData for downstream analysis (next step = analyses/KI_Compartment_filter_samples.R)

In [ ]:
# Build the phylogenetic tree to add to phyloseq object

#Originally from Moorea_Sym pipeline
#data/Bioinf/clust/all_rep_set_rep_set.fasta = seqs
#data/tax_table.txt = ISDs
#column 9 is clade ID
#awk '{print $1}' data/tax_table.txt = denovo name
#awk '{print $8}' data/tax_table.txt = clade name

#subset ID lists by clade
awk '$9 ~ /^A/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/A_tree_seq.ids
awk '$9 ~ /^B/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/B_tree_seq.ids
awk '$9 ~ /^C/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/C_tree_seq.ids
awk '$9 ~ /^D/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/D_tree_seq.ids
awk '$9 ~ /^E/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/E_tree_seq.ids
awk '$9 ~ /^F/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/F_tree_seq.ids
awk '$9 ~ /^G/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/G_tree_seq.ids
awk '$9 ~ /^H/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/H_tree_seq.ids
awk '$9 ~ /^I/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/I_tree_seq.ids

#filter seqs by id list
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/A_tree_seq.ids -o data/Bioinf/tree/A_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/B_tree_seq.ids -o data/Bioinf/tree/B_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/C_tree_seq.ids -o data/Bioinf/tree/C_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/D_tree_seq.ids -o data/Bioinf/tree/D_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/E_tree_seq.ids -o data/Bioinf/tree/E_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/F_tree_seq.ids -o data/Bioinf/tree/F_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/G_tree_seq.ids -o data/Bioinf/tree/G_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/H_tree_seq.ids -o data/Bioinf/tree/H_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/I_tree_seq.ids -o data/Bioinf/tree/I_tree_seqs.fasta

#check seq number
grep -c ">" data/Bioinf/tree/A_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/B_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/C_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/D_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/E_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/F_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/G_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/H_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/I_tree_seqs.fasta

#align fasta files for each clade
align_seqs.py -i data/Bioinf/tree/A_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/B_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/C_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/D_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/E_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/F_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/G_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/H_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/I_tree_seqs.fasta -m muscle -o data/Bioinf/tree/

#remove extra header info from alignments
sed 's/ .*//' data/Bioinf/tree/A_tree_seqs_aligned.fasta > data/Bioinf/tree/A_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/B_tree_seqs_aligned.fasta > data/Bioinf/tree/B_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/C_tree_seqs_aligned.fasta > data/Bioinf/tree/C_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/D_tree_seqs_aligned.fasta > data/Bioinf/tree/D_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/E_tree_seqs_aligned.fasta > data/Bioinf/tree/E_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/F_tree_seqs_aligned.fasta > data/Bioinf/tree/F_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/G_tree_seqs_aligned.fasta > data/Bioinf/tree/G_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/H_tree_seqs_aligned.fasta > data/Bioinf/tree/H_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/I_tree_seqs_aligned.fasta > data/Bioinf/tree/I_tree_seqs_aligned_clean.fasta

In [ ]:
# Format and filter the RData file
R --vanilla < analyses/KI_Compartment_filter_samples.R
# Input file is analyses/KI_Compartment.RData
# Must run build_phy_tree first for this to work properly
# Output file is: data/KI_seqs_f_coral_grouped.RData

In [ ]:
# Rarefaction
# Multiple rarefactions to look at different sequence rarefaction depths
# 1000 to 10000 in steps of 1000
multiple_rarefactions.py -i data/Bioinf/clust/97_otus_bysample.biom -o data/Bioinf/clust/rarefy/mult -m 1000 -x 10000 -s 1000
# 200 to 1000 in steps of 100
multiple_rarefactions.py -i data/Bioinf/clust/97_otus_bysample.biom -o data/Bioinf/clust/rarefy/mult -m 200 -x 1000 -s 100

# Single rarefaction to 1000 seqs
single_rarefaction.py -i data/Bioinf/clust/97_otus_bysample.biom -o data/Bioinf/clust/rarefy/rarefaction_1000.biom --depth 1000
# Convert .biom to tsv
biom convert -i data/Bioinf/clust/rarefy/rarefaction_1000.biom -o data/Bioinf/clust/rarefy/rarefaction_1000.tsv --to-tsv
# Build a phyloseq object
R --vanilla < ~/SymITS2/build_phyloseq.R --args data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv data/Coralphoto__Metadata/KI_Compartment_metadata.tsv data/Bioinf/clust/rarefy/rarefaction_1000.tsv data/ITS2db_trimmed_notuniques_otus.txt analyses/KI_Compartment_rarefy.RData
# Arguments are 1) nw_tophits - taxonomic assignment output from run_nw.R, 2) sample metadata in tsv format, 3) OTU table, 4) duplicate reference taxa names, 5) output filename
# Creates analyses/KI_Compartment.RData

# Filter out sequences which do not blast to Symbiodnium
R --vanilla < ~/SymITS2/filter_notsym.R --args analyses/KI_Compartment_rarefy.RData data/Bioinf/clust/all_rep_set_rep_set.fasta analyses/KI_Compartment_rarefy.RData /media/danielle/CoralDrive/blastdb

# Build the phylogenetic tree to add to phyloseq object
chmod u+x data/Bioinf/tree/build_phy_tree_rare.sh # Make sure script can run
data/Bioinf/tree/build_phy_tree_rare.sh # Run build_phy_tree script

# Format and filter the RData file
R --vanilla < analyses/KI_Compartment_filter_samples_rare.R